In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_excel("omo_sales_data.xlsx")
df1.head(5)

,transaction_id,date,customer_id,category,product,quantity,unit_price,total_price,total_cost,gross_profit,profit_margin,payment_method
0,TXN000000,2023-01-01 00:01:00,CUST14629,Health Supplies,Amoxicillin 25mg,1,23.41,23.41,8.99,14.42,61.60,Debit Card
1,TXN000001,2023-01-01 00:01:00,CUST38699,Health Supplies,Doxycycline 10mg,3,15.91,47.73,23.97,23.76,49.78,Credit Card
2,TXN000002,2023-01-01 00:02:00,CUST15248,Health Supplies,Hydrocortisone Cream (Generic Hydrocortisone),1,9.17,9.17,4.49,4.68,51.04,Debit Card
3,TXN000003,2023-01-01 00:02:00,CUST46926,Health Supplies,Facial Cleanser (CeraVe),3,7.18,21.54,10.47,11.07,51.39,Debit Card
4,TXN000004,2023-01-01 00:02:00,CUST00426,Health Supplies,Hydrocortisone Cream,1,11.69,11.69,4.49,7.20,61.59,Cash


In [3]:
df2 = df1.drop(['transaction_id',	'customer_id', 'category','profit_margin', 'payment_method'], axis='columns')
df2['Year'] = df2['date'].dt.year
df2.head()

,date,product,quantity,unit_price,total_price,total_cost,gross_profit,Year
0,2023-01-01 00:01:00,Amoxicillin 25mg,1,23.41,23.41,8.99,14.42,2023
1,2023-01-01 00:01:00,Doxycycline 10mg,3,15.91,47.73,23.97,23.76,2023
2,2023-01-01 00:02:00,Hydrocortisone Cream (Generic Hydrocortisone),1,9.17,9.17,4.49,4.68,2023
3,2023-01-01 00:02:00,Facial Cleanser (CeraVe),3,7.18,21.54,10.47,11.07,2023
4,2023-01-01 00:02:00,Hydrocortisone Cream,1,11.69,11.69,4.49,7.20,2023


In [4]:
df2.describe()

,date,quantity,unit_price,total_price,total_cost,gross_profit,Year
count,500000,500000.00000,500000.000000,500000.000000,500000.000000,500000.000000,500000.0
mean,2023-07-02 14:28:34.357679360,2.00051,19.350443,38.707759,15.651071,23.056688,2023.0
min,2023-01-01 00:01:00,1.00000,2.990000,2.990000,1.490000,1.500000,2023.0
25%,2023-04-02 06:35:00,1.00000,13.440000,20.040000,7.990000,11.910000,2023.0
50%,2023-07-02 20:05:30,2.00000,18.090000,33.420000,13.470000,19.580000,2023.0
75%,2023-10-01 16:54:15,3.00000,23.430000,51.140000,20.970000,30.400000,2023.0
max,2023-12-31 23:59:00,3.00000,59.990000,179.940000,53.970000,125.970000,2023.0
std,NaN,0.81647,8.661631,24.478820,10.088080,14.950712,0.0


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

X = df2[['product', 'quantity', 'unit_price','total_cost','total_price','Year']]
y = df2['gross_profit']

categorical_cols = ['product']
numerical_cols = ['quantity', 'unit_price','total_cost','total_price','Year']

column_transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols)], remainder='passthrough')

X = column_transformer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)
dt_model.score(X_test, y_test)

0.999999872712881

In [12]:
new_data = {
    'product': 'Facial Cleanser (CeraVe)',
    'quantity': 6,
    'unit_price': 93.41,
    'total_cost': 48.67,
    'total_price': 3.41,
    'Year': 2024
}
new_df = pd.DataFrame([new_data])
new_data_transformed = column_transformer.transform(new_df)
pred = dt_model.predict(new_data_transformed)
print(pred[0])

1.93


In [9]:
import pickle
pickle.dump(dt_model, open('project_model.pkl', 'wb'))

In [10]:
pickle.dump(column_transformer, open('column_transformer.pkl', 'wb'))

In [7]:
product_list = df1['product'].drop_duplicates().sort_values().tolist()
import json
# Save the list to a JSON file
with open('product_list.json', 'w') as json_file:
    json.dump(product_list, json_file)